**Carrega as bibliotecas e testa a conexão com banco de dados PostGreeSQL**

In [6]:
#Bibliotecas Python
import sys
import os
import pandas as pd
from pathlib import Path
from sqlalchemy import text

#Bibliotecas Proprias
if os.path.abspath(os.path.join(os.getcwd(), '..')) not in sys.path: sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import src.connection as cn  # banco PostGreSQL

#Teste de conexao
cn.connection_test()

✅ Sucesso: Conexão estabelecida com o banco de dados.


**Executa consultas SQL e retorna resultado em um dataframe PANDAS**

---
**1. Análise Temporal (linha do tempo)**

Estamos observando a **evolução histórica do faturamento total** da operação, consolidada cronologicamente mês a mês. O resultado apresenta o montante financeiro bruto gerado em cada período (agrupado por Ano e Mês), calculado através da soma do preço dos produtos vendidos acrescido dos seus respectivos custos de frete, o que permite analisar tendências de crescimento, picos de receita e a sazonalidade das vendas ao longo do tempo.

In [15]:
#SUM(price + freight_value) → soma o valor pago pelo produto + frete.
#A tabela dim_date fornece ano e mês, permitindo o agrupamento mensal.
#GROUP BY organiza os resultados por ano e mês.

session = cn.get_session().bind
consulta = """
SELECT
    d.year,
    d.month,
    SUM(f.price + f.freight_value) AS total_vendas
FROM fact_sales f
JOIN dim_date d 
    ON f.date_sk = d.date_sk
GROUP BY 
    d.year,
    d.month
ORDER BY 
    d.year,
    d.month;
"""
df = pd.read_sql(sql=text(consulta), con=session)
df.head(12)

,year,month,total_vendas
0,2016,9,354.75
1,2016,10,56808.84
2,2016,12,19.62
3,2017,1,137188.49
4,2017,2,286280.62
5,2017,3,432048.59
6,2017,4,412422.24
7,2017,5,586190.95
8,2017,6,502963.04
9,2017,7,584971.62


---
**2. Ranking / TOP N (os melhores/piores)**

Estamos observando um **ranking dos 10 produtos específicos mais vendidos** em termos de volume transacional (frequência de vendas). A consulta agrupa os dados da tabela fato pela chave do produto (`product_sk`) e contabiliza quantas vezes cada item aparece nos registros de venda, resultando em uma lista dos itens individuais com maior saída ("best-sellers"), ordenados de forma decrescente pela quantidade vendida. Note que, diferentemente do comentário no código, o agrupamento é feito pelo **ID do produto** e não pela **categoria**, portanto o resultado mostrará itens específicos.

In [14]:
#As 10 categorias com maior quantidade de vendas.
#COUNT(product_sk) conta cada venda registrada na fact table.
#Ordena do maior para o menor.

session = cn.get_session().bind
consulta = text("""
SELECT
    f.product_sk,
    COUNT(f.product_sk) AS quantidade_vendida
FROM fact_sales f
GROUP BY
    f.product_sk
ORDER BY
    quantidade_vendida DESC
LIMIT 10;
""")
df = pd.read_sql(sql=consulta, con=session)
df.head(10)

,product_sk,quantidade_vendida
0,32681,527
1,16638,488
2,12403,484
3,4066,392
4,32525,388
5,7185,373
6,25322,343
7,23313,323
8,3207,281
9,30641,274


---
**3. Agregação Multidimensional (várias perspectivas)**

Estamos observando uma **matriz de vendas completa e multidimensional** que consolida o faturamento sob todas as combinações de perspectivas possíveis simultaneamente. Graças ao operador `CUBE`, o resultado não se limita aos dados detalhados (quanto cada vendedor faturou com cada produto em um mês específico), mas gera automaticamente todos os níveis de **subtotais e agregações hierárquicas** — como o total acumulado apenas por vendedor, o desempenho isolado de um produto, o faturamento fechado por ano ou mês — culminando no **Total Geral** da operação (onde as dimensões aparecem como `NULL`), permitindo uma análise flexível de "quem", "o quê" e "quando" em uma única estrutura de dados.

In [18]:
#O GROUP BY CUBE vai gerar automaticamente:
#✔ Detalhamento máximo:
#seller_sk + product_sk + ano + mês
#✔ Totais parciais:
#seller_sk + product_sk
#seller_sk + mês
#product_sk + mês
#somente seller_sk
#somente product_sk
#somente período (year, month)
#✔ Total geral:
#todos os campos NULL

session = cn.get_session().bind
consulta = text("""
SELECT
    f.seller_sk,
    f.product_sk,
    d.year,
    d.month,
    SUM(f.price + f.freight_value) AS total_vendas
FROM fact_sales f
JOIN dim_date d
    ON f.date_sk = d.date_sk
GROUP BY CUBE (
    f.seller_sk,
    f.product_sk,
    d.year,
    d.month
)
ORDER BY
    f.seller_sk NULLS LAST,
    f.product_sk NULLS LAST,
    d.year NULLS LAST,
    d.month NULLS LAST;
""")
df = pd.read_sql(sql=consulta, con=session)
df.head(10)

,seller_sk,product_sk,year,month,total_vendas
0,1.0,24485.0,2018.0,5.0,38.37
1,1.0,24485.0,2018.0,NaN,38.37
2,1.0,24485.0,NaN,5.0,38.37
3,1.0,24485.0,NaN,NaN,38.37
4,1.0,NaN,2018.0,5.0,38.37
5,1.0,NaN,2018.0,NaN,38.37
6,1.0,NaN,NaN,5.0,38.37
7,1.0,NaN,NaN,NaN,38.37
8,2.0,5919.0,2017.0,6.0,305.98
9,2.0,5919.0,2017.0,NaN,305.98


---
**4. Análise de Cohort / Retenção (comportamento ao longo do tempo)**

Estamos observando uma **Análise de Coorte (Cohort Analysis)** focada na retenção de clientes, que segmenta os consumidores com base no mês de sua primeira compra e rastreia o comportamento de recompra ao longo do tempo. A consulta calcula o ciclo de vida de cada grupo (coorte), medindo quantos clientes retornaram em meses subsequentes (o "month_number" ou deslocamento) em relação ao tamanho original do grupo, resultando em uma matriz que expõe a **taxa de fidelidade** percentual e absoluta, permitindo identificar se os clientes continuam engajados ou abandonam a marca após a primeira aquisição.

In [21]:
#-- Resumo da Análise de Cohort/Retenção:
#1. IDENTIFICAÇÃO DA COORTE: Define o mês da primeira compra (coorte) para cada cliente.
#2. CÁLCULO DO DESLOCAMENTO: Calcula a diferença em meses entre o mês de cada compra e o mês da coorte.
#3. CÁLCULO DA RETENÇÃO: Agrupa o resultado para contar clientes retidos e calcular a taxa de retenção ao longo do tempo.

session = cn.get_session().bind
consulta = text("""
WITH Customer_First_Order AS (
    -- 1. Encontra a data da primeira compra e trunca para o início do mês (Coorte)
    SELECT
        t1.customer_sk,
        -- CAST para garantir que a DB trate a coluna como TIMESTAMP
        DATE_TRUNC('month', MIN(CAST(t2.order_purchase_timestamp AS TIMESTAMP))) AS cohort_start_date
    FROM
        fact_sales AS t1
    JOIN
        df_orders AS t2 ON t1.order_id = t2.order_id
    GROUP BY
        t1.customer_sk
),
Customer_Activity AS (
    -- 2. Combina o mês da coorte com o mês de cada compra subsequente e calcula a diferença em meses
    SELECT
        cfo.customer_sk,
        TO_CHAR(cfo.cohort_start_date, 'YYYY-MM') AS cohort_month, -- Mês da Primeira Compra (Coorte formatado)
        
        -- Calcula o número de meses entre o início do mês da compra atual e o início do mês da coorte.
        (
            (EXTRACT(YEAR FROM DATE_TRUNC('month', CAST(t2.order_purchase_timestamp AS TIMESTAMP))) - EXTRACT(YEAR FROM cfo.cohort_start_date)) * 12 +
            (EXTRACT(MONTH FROM DATE_TRUNC('month', CAST(t2.order_purchase_timestamp AS TIMESTAMP))) - EXTRACT(MONTH FROM cfo.cohort_start_date))
        ) AS month_number -- Número de meses desde a coorte (0 = Mês da Coorte)
    FROM
        Customer_First_Order AS cfo
    JOIN
        fact_sales AS t1 ON cfo.customer_sk = t1.customer_sk
    JOIN
        df_orders AS t2 ON t1.order_id = t2.order_id
    GROUP BY
        cfo.customer_sk,
        cfo.cohort_start_date,
        -- Agrupar pelo início do mês da compra atual (com CAST)
        DATE_TRUNC('month', CAST(t2.order_purchase_timestamp AS TIMESTAMP))
)
-- 3. Calcula o número de clientes retidos e a taxa de retenção
SELECT
    ca.cohort_month,
    ca.month_number,
    COUNT(DISTINCT ca.customer_sk) AS retained_customers,
    -- Calcula o tamanho inicial da coorte (clientes no Month 0)
    (
        SELECT
            COUNT(DISTINCT customer_sk)
        FROM
            Customer_Activity
        WHERE
            cohort_month = ca.cohort_month AND month_number = 0
    ) AS cohort_size,
    -- Calcula a taxa de retenção (Retained / Cohort Size)
    CAST(COUNT(DISTINCT ca.customer_sk) AS REAL) * 100 /
    (
        SELECT
            COUNT(DISTINCT customer_sk)
        FROM
            Customer_Activity
        WHERE
            cohort_month = ca.cohort_month AND month_number = 0
    ) AS retention_rate
FROM
    Customer_Activity AS ca
GROUP BY
    ca.cohort_month,
    ca.month_number
ORDER BY
    ca.cohort_month,
    ca.month_number;
""")
df = pd.read_sql(sql=consulta, con=session)
df.head(10)



,cohort_month,month_number,retained_customers,cohort_size,retention_rate
0,2016-09,0.0,3,3,100.0
1,2016-10,0.0,308,308,100.0
2,2016-12,0.0,1,1,100.0
3,2017-01,0.0,789,789,100.0
4,2017-02,0.0,1733,1733,100.0
5,2017-03,0.0,2641,2641,100.0
6,2017-04,0.0,2391,2391,100.0
7,2017-05,0.0,3660,3660,100.0
8,2017-06,0.0,3217,3217,100.0
9,2017-07,0.0,3969,3969,100.0


---
**5. KPI (indicador-chave de negócio)**

Estamos analisando a evolução histórica do **Ticket Médio Mensal**, um indicador financeiro que revela o valor médio gasto pelos clientes em cada transação ao longo do tempo. A consulta opera consolidando primeiramente os itens de venda para calcular o valor total de cada pedido individual (somando preço e frete) e, em seguida, agrupa esses dados cronologicamente para dividir o faturamento total do mês pela quantidade de pedidos únicos, permitindo identificar tendências de valorização das vendas ou mudanças no comportamento de compra.

In [23]:
#Análise: Ticket Médio Mensal
#1. AGREGAÇÃO POR PEDIDO: Soma 'price' e 'freight_value' para calcular a receita total por order_id.
#2. AGREGAÇÃO POR MÊS: Agrupa os pedidos pelo mês de compra.
#3. CÁLCULO FINAL: Divide a Receita Total pela Contagem de Pedidos Distintos (Ticket Médio).

session = cn.get_session().bind
consulta = text("""
WITH Order_Revenue AS (
    -- 1. Calcula a Receita Total (Preço + Frete) para CADA Pedido (Order_ID)
    SELECT
        t1.order_id,
        -- Conversão explícita para TIMESTAMP para evitar erro de tipo de dados
        CAST(t2.order_purchase_timestamp AS TIMESTAMP) AS order_timestamp,
        SUM(t1.price + t1.freight_value) AS total_order_revenue
    FROM
        fact_sales AS t1
    JOIN
        df_orders AS t2 ON t1.order_id = t2.order_id
    GROUP BY
        t1.order_id,
        t2.order_purchase_timestamp -- Agrupamento para somar os itens de um mesmo pedido
)
-- 2. Calcula o Ticket Médio por Mês (Receita Total / Número de Pedidos Distintos)
SELECT
    -- Trunca a data para o início do mês e formata para 'YYYY-MM'
    TO_CHAR(DATE_TRUNC('month', order_timestamp), 'YYYY-MM') AS order_month, 
    COUNT(DISTINCT order_id) AS total_orders, 
    SUM(total_order_revenue) AS total_revenue, 
    -- Ticket Médio = (Soma da Receita) / (Contagem Distinta de Pedidos)
    ROUND(CAST(SUM(total_order_revenue) AS NUMERIC) / COUNT(DISTINCT order_id), 2) AS average_ticket
FROM
    Order_Revenue
GROUP BY
    order_month
ORDER BY
    order_month;
""")
df = pd.read_sql(sql=consulta, con=session)
df.head(10)

,order_month,total_orders,total_revenue,average_ticket
0,2016-09,3,354.75,118.25
1,2016-10,308,56808.84,184.44
2,2016-12,1,19.62,19.62
3,2017-01,789,137188.49,173.88
4,2017-02,1733,286280.62,165.19
5,2017-03,2641,432048.59,163.59
6,2017-04,2391,412422.24,172.49
7,2017-05,3660,586190.95,160.16
8,2017-06,3217,502963.04,156.35
9,2017-07,3969,584971.62,147.39
